In [3]:
# https://pypi.org/project/clean-text/
!pip install clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.5 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=b5a2d314be99bf29037c3fc216823b6fa31bcddc23f661e7732f88352808cc67
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [4]:
import pandas as pd
import numpy as np

from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

from cleantext import clean

In [5]:
#
import nltk
from nltk.corpus import stopwords

# разбитие текста на предложения, слова, токены
nltk.download('punkt')
# стоп-слова. которые не несут нагрузки: предлоги, например
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
sw = stopwords.words('russian')
sw

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [12]:
path = r'/content/categorised_posts_1696056372.csv'
df = pd.read_csv(path)
df = df.dropna(subset=['content'])

df = df.replace(['РљРЅРёРіРё'], 'Книги')
df = df.replace(['РљРёРЅРѕ'], 'Кино')

df = df[['content', 'category_ru']]
df = df.rename(columns={"content": "text"})
df.head()

,text,category_ru
0,Путин потребовал разобраться с ростом цен на т...,Авто
1,Skoda показала новое поколение кроссовера Kodi...,Авто
2,В России цены на новые автомобили продолжают б...,Авто
3,АвтоВАЗ запустил онлайн продажи автомобилей ...,Авто
4,В августе российские заводы произвели 50 тыс....,Авто


In [13]:
# текущий набор тем и категорий
domain_tags = {
    'Авто': 'Транспорт',
    'Банки и кредиты': 'Финансы',
    'Беременность и роды': 'Семья и дети',
    'Билеты': 'Отдых и путешествия',
    'Бухгалтерские услуги': 'Бизнес',
    'Бытовая техника': 'Бытовая техника',
    'Грузоперевозки и транспортные услуги': 'Бизнес',
    'Дизайн интерьера': 'Строительство, обустройство и ремонт',
    'Домашние животные': 'Животные',
    'Дополнительное образование и курсы': 'Образование',
    'Доставка воды': 'Еда и напитки',
    'Доставка готовых блюд и продуктов': 'Еда и напитки',
    'Доступ в интернет и мобильная связь': 'Телеком',
    'Зарубежный туризм': 'Отдых и путешествия',
    'Здоровое питание': 'Красота и здоровье',
    'Игры': 'Развлечения и досуг',
    'Инвестиции': 'Финансы',
    'Канцелярские товары': 'Канцелярские товары',
    'Кино': 'Развлечения и досуг',
    'Книги': 'Развлечения и досуг',
    'Компьютерная техника': 'Электроника',
    'Кулинария': 'Еда и напитки',
    'Мебель': 'Строительство, обустройство и ремонт',
    'Медицина': 'Красота и здоровье',
    'Мото': 'Транспорт',
    'Музыка': 'Развлечения и досуг',
    'Одежда и обувь': 'Одежда, обувь и аксессуары',
    'Охота': 'Развлечения и досуг',
    'Парфюмерия и косметика': 'Красота и здоровье',
    'Подарки и цветы': 'Подарки и цветы',
    'Покупка и аренда': 'Недвижимость',
    'Принтеры и МФУ': 'Электроника',
    'Прокат автомобилей': 'Отдых и путешествия',
    'Работа': 'Работа',
    'Ремонт': 'Строительство, обустройство и ремонт',
    'Рестораны': 'Развлечения и досуг',
    'Российский туризм': 'Отдых и путешествия',
    'Рыбалка': 'Развлечения и досуг',
    'Свадьба': 'Семья и дети',
    'Сельскохозяйственное оборудование и техника': 'Сельскохозяйственное оборудование и техника',
    'Смартфоны и гаджеты': 'Электроника',
    'Создание и продвижение сайтов': 'Бизнес',
    'Спорт': 'Спорт',
    'Танцы': 'Развлечения и досуг',
    'Театры': 'Развлечения и досуг',
    'Телевидение': 'Развлечения и досуг',
    'Товары для детей': 'Семья и дети',
    'Товары для дома': 'Строительство, обустройство и ремонт',
    'Форекс': 'Финансы',
    'Фото, видео и аудиотехника': 'Электроника',
    'Школа и вуз': 'Образование',
    'Юридические услуги': 'Бизнес'
}

In [15]:
target_tags = dict()
categories = sorted(list(df.category_ru.unique()))

for i in range(len(categories)):
    target_tags[categories[i]] = i
target_tags

{'Авто': 0,
 'Банки и кредиты': 1,
 'Беременность и роды': 2,
 'Билеты': 3,
 'Бухгалтерские услуги': 4,
 'Бытовая техника': 5,
 'Грузоперевозки и транспортные услуги': 6,
 'Дизайн интерьера': 7,
 'Домашние животные': 8,
 'Дополнительное образование и курсы': 9,
 'Доставка воды': 10,
 'Доставка готовых блюд и продуктов': 11,
 'Доступ в интернет и мобильня связь': 12,
 'Зарубежный туризм': 13,
 'Здоровое питание': 14,
 'Игры': 15,
 'Инвестиции': 16,
 'Канцелярские товары': 17,
 'Кино': 18,
 'Книги': 19,
 'Компьютерная техника': 20,
 'Кулинария': 21,
 'Мебель': 22,
 'Медицина': 23,
 'Мото': 24,
 'Музыка': 25,
 'Одежда и обувь': 26,
 'Охота': 27,
 'Парфюмерия и косметика': 28,
 'Подарки и цветы': 29,
 'Покупка и аренда': 30,
 'Принтеры и МФУ': 31,
 'Прокат автомобилей': 32,
 'Работа': 33,
 'Ремонт': 34,
 'Рестораны': 35,
 'Российский туризм': 36,
 'Рыбалка': 37,
 'Свадьба': 38,
 'Сельскохозяйственное оборудование и техника': 39,
 'Смартфоны и гаджеты': 40,
 'Создание и продвижение сайтов':

In [17]:
# собираем из разных (сейчас 1) наборов один полноценный
df_res = pd.concat([
    df
                    ])

In [18]:
# обработка датасета на длину строк: вычилить длину и оставить только больше 30 символов
df_res['text_legth'] = df_res['text'].apply(lambda x: len(x) if type(x)==str else 0)
df_res = df_res[(df_res['text_legth']>30)]

# очситка датасета: удаление эмодзи и применение to_ascii=False
df_res['target'] = df_res['category_ru'].apply(lambda x: target_tags[x])
df_res['text_clean'] = df_res['text'].apply(lambda x: clean(x, no_emoji=True, to_ascii=False))

# перемешать, чтобы в датасете были данные ранломно расположены, а не по порядку
df_res = shuffle(df_res, random_state=2024)

<ipython-input-18-00d11df3f9a4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['target'] = df_res['category_ru'].apply(lambda x: target_tags[x])


In [19]:
# определение тестовой и тренирочной выборки
X = df_res['text_clean']
y = df_res['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [20]:
# Определение важности слов
'''
TF-IDF вычисляет важность каждого слова в документе относительно количества его употреблений
в данном документе и во всей коллекции текстов.
Этот метод позволяет выделить ключевые слова и понять,
какие слова имеют больший вес для определенного документа в контексте всей коллекции.
'''
vect_word = TfidfVectorizer(
    max_features=6000,
    lowercase=True,
    analyzer="word",
    stop_words=sw,
    ngram_range=(1, 7),
    dtype=np.float32
)

In [21]:
# Train tf-idf
X_train_tfidf = vect_word.fit_transform(X_train)

# Map tf-idf on test
X_test_tfidf = vect_word.transform(X_test)

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
# Init logreg model
logreg = LogisticRegression(
    C=2.5, #Обратная сила регуляризации;Выбор оптимального значения C часто осуществляется с использованием методов кросс-валидации
    random_state=42,
    solver='saga'#определяет алгоритм, используемый для оптимизации функции потерь
)

#### Инициализация и обучение модели логистической регрессии

In [24]:
# Train logreg
logreg.fit(X_train_tfidf, y_train)

LogisticRegression(C=2.5, random_state=42, solver='saga')

#### Предсказание вероятностей и классов на тестовых данных:

In [25]:
# Predict probabilities
preds_proba = logreg.predict_proba(X_test_tfidf)
# Get classes
preds = np.argmax(preds_proba, axis=1)

In [26]:
print(classification_report(y_test, preds, target_names=categories))

                                             precision    recall  f1-score   support

                                       Авто       0.84      0.71      0.77       124
                            Банки и кредиты       0.71      0.59      0.64       177
                        Беременность и роды       0.72      0.63      0.67        99
                                     Билеты       0.91      0.78      0.84       139
                       Бухгалтерские услуги       0.70      0.85      0.76       534
                            Бытовая техника       0.92      0.69      0.79        88
       Грузоперевозки и транспортные услуги       0.98      0.76      0.86        79
                           Дизайн интерьера       0.77      0.58      0.66        81
                          Домашние животные       0.83      0.34      0.49        29
         Дополнительное образование и курсы       0.67      0.65      0.66       236
                              Доставка воды       0.83      0.58

#### Импортируем модельку

In [27]:
import pickle

# Сохранение TF-IDF векторизатора:
pickle.dump(vect_word, open('tf_idf_word_vectorizer', 'wb')) # pickling
# Сохранение модели логистической регрессии:
pickle.dump(logreg, open('logreg_classifier', 'wb')) # pickling